In [49]:
from scrapy import Selector
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd

In [51]:
def scrape_wellcee_listings():

    driver = webdriver.Chrome(ChromeDriverManager().install())

    driver.get("https://www.wellcee.com/rent-apartment/shanghai/list?cityId=15102233103895305#rentTypeIds=15103931190241306")
    selenium_response = driver.page_source

    new_selector = Selector(text=selenium_response)

    items = new_selector.xpath('//*[@class="house-item clearfix"]/a/@href').extract()

    items = list(set(['https://www.wellcee.com'+listing_id for listing_id in items]))

    return items

In [52]:
def scrape_wellcee_data(listing_list):
    appended_data = []

    counter = 0

    for url in listing_list:
        try:
            apartment_dict = {}

            text_response = requests.get(url).text
            sel = Selector(text = text_response)

            apartment_dict['url'] = url
            apartment_dict['price'] = sel.xpath('//p[@class = "price"]/text()').extract()[0].strip('Rent : ').strip('RMB/M')
            apartment_dict['location'] = sel.xpath('//p[@class = "house-address"]/text()').extract()[0]
            apartment_dict['district'] = sel.xpath('//span[@class = "breadcrumb-item"]/a/text()').extract()[2]
            apartment_dict['size'] = sel.xpath('//div[@class = "list clearfix"]/div/span/text()').extract()[7].strip('㎡')
            apartment_dict['floor'] = sel.xpath('//div[@class = "list clearfix"]/div/span/text()').extract()[11]

            appended_data.append(apartment_dict)

            counter = counter+1
    
            if counter%15 == 0:
                print('Aggregated listings: {}.'.format(counter), end = ' ')

        except:
            pass

    df = pd.DataFrame(appended_data)

    df = df.reset_index().rename(columns = {'index':'id'})

    return df

In [53]:
wellcee_listings = scrape_wellcee_listings()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/ilya/.wdm/drivers/chromedriver/mac64/98.0.4758.48/chromedriver] found in cache
ipykernel_launcher:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


In [54]:
df = scrape_wellcee_data(wellcee_listings)

Aggregated listings: 15. 

In [55]:
df

,id,url,price,location,district,size,floor
0,0,https://www.wellcee.com/rent-apartment/1639628...,27800,Shanghai Changning 南洋新华公寓,Changning,150,14
1,1,https://www.wellcee.com/rent-apartment/1641450...,5500,Shanghai Jing‘An 共和新路街道中山北路835号,Jing‘An,32,3
2,2,https://www.wellcee.com/rent-apartment/1631710...,2700,Shanghai Baoshan 华秋路58弄32号,Baoshan,25,7
3,3,https://www.wellcee.com/rent-apartment/1640170...,7600,Shanghai Jing‘An 487 Yuyao Road building 87 ro...,Jing‘An,30,4
4,4,https://www.wellcee.com/rent-apartment/1582454...,8000,Shanghai Xuhui 襄阳南路500号,Xuhui,30,26
5,5,https://www.wellcee.com/rent-apartment/1642845...,2000,Shanghai Minhang 南华街25号 泊寓青年家,Minhang,25,2
6,6,https://www.wellcee.com/rent-apartment/1641308...,10800,Shanghai Pudong 江高路105弄江悦名庭1号楼,Pudong,57,5
7,7,https://www.wellcee.com/rent-apartment/1643443...,4500,Shanghai Pudong 汇滕南苑 瑞浦路612弄 18号线沈梅路！！,Pudong,80,5
8,8,https://www.wellcee.com/rent-apartment/1641229...,3700,Shanghai Jing‘An 凤阳路652号,Jing‘An,37,3
9,9,https://www.wellcee.com/rent-apartment/1639460...,5999,Shanghai Xuhui 徐家汇地铁站,Xuhui,60,3


In [38]:
appended_data = []
counter = 0

for url in wellcee_listings:

    apartment_dict = {}

    text_response = requests.get(url).text
    sel = Selector(text = text_response)

    apartment_dict['url'] = url
    apartment_dict['price'] = sel.xpath('//p[@class = "price"]/text()').extract()[0].strip('Rent : ').strip('RMB/M')
    apartment_dict['location'] = sel.xpath('//p[@class = "house-address"]/text()').extract()[0]
    apartment_dict['district'] = sel.xpath('//span[@class = "breadcrumb-item"]/a/text()').extract()[2]
    apartment_dict['size'] = sel.xpath('//div[@class = "list clearfix"]/div/span/text()').extract()[7].strip('㎡')
    apartment_dict['floor'] = sel.xpath('//div[@class = "list clearfix"]/div/span/text()').extract()[11]

    appended_data.append(apartment_dict)

    counter = counter+1
    
    if counter%15 == 0:
        print('Aggregated listings: {}.'.format(counter), end = ' ')

Aggregated listings: 15. 

In [39]:
df = pd.DataFrame(appended_data)

In [40]:
df

,url,price,location,district,size,floor
0,https://www.wellcee.com/rent-apartment/1641800...,12000,Shanghai Hongkou 瑞虹路383号,Hongkou,102,5
1,https://www.wellcee.com/rent-apartment/1643263...,5000,Shanghai Huangpu The apto is in tianzifang in ...,Huangpu,10m²,2
2,https://www.wellcee.com/rent-apartment/1582454...,8000,Shanghai Xuhui 襄阳南路500号,Xuhui,30,26
3,https://www.wellcee.com/rent-apartment/1639112...,5600,Shanghai Huangpu 制造局路833弄7号楼,Huangpu,32,5
4,https://www.wellcee.com/rent-apartment/1643333...,4900,Shanghai Xuhui 太原路237号,Xuhui,30,1
5,https://www.wellcee.com/rent-apartment/1636707...,4700,Shanghai Huangpu 黄浦区制造局路833弄26号,Huangpu,15,5
6,https://www.wellcee.com/rent-apartment/1635920...,4294,Shanghai Pudong 科苑路99号,Pudong,25,1
7,https://www.wellcee.com/rent-apartment/1643533...,16500,Shanghai Jing‘An 武定路1128弄,Jing‘An,110,17
8,https://www.wellcee.com/rent-apartment/1642825...,11500,Shanghai Xuhui 五原路281,Xuhui,85,1
9,https://www.wellcee.com/rent-apartment/1641365...,6500,Shanghai Xuhui 肇嘉浜路669弄,Xuhui,30,2


In [23]:
text_response = requests.get('https://www.wellcee.com/rent-apartment/16434432271098164').text

In [24]:
sel = Selector(text = text_response)

In [27]:
sel.xpath('//p[@class = "price"]/text()').extract()[0].strip('Rent : ').strip('RMB/M')

'2815'

In [54]:
sel.xpath('//p[@class = "house-address"]/text()').extract()[0]

'Shanghai Xuhui 凯旋南路31号'

In [67]:
sel.xpath('//span[@class = "breadcrumb-item"]/a/text()').extract()[2]

'Xuhui'

In [29]:
sel.xpath('//div[@class = "list clearfix"]/div/span/text()').extract()[7].strip('㎡')

'15'

In [77]:
sel.xpath('//div[@class = "list clearfix"]/div/span/text()').extract()[11]

'2'